## Are my twin brother and I actually identical twins? 
   
Ever since we were born my twin brother Paul and I have been told that we were
fraternal twins. However, recently my mother has taken up genealogy as a hobby 
and had the entire family do a DNA test using the FamilyTreeDNA (FTDNA). The DNA testing services presents the results of the tests in terms of shared [**centimorgans**](https://en.wikipedia.org/wiki/Centimorgan) with other users of their service.  For geneaological purposes this is sufficient as it allows the service to make an imperfect guess regarding the relationship of two people [(they often use other factors such as self-reported age in these estimates)](https://slate.com/technology/2019/10/23andme-family-secrets-half-siblings-cousins.html). [ISSOG](https://isogg.org/wiki/Autosomal_DNA_statistics) provides a nifty chart of the relationship between centimorgans and relationship:    

<img src="img/Shared_cM.jpg">

The problem is that the highest match, for where people share approximately 50% of their DNA, is parent/child.  This is how my brother and I are shown to be related.  What I want to confirm is that we are indeed fully identical.  Fortunately, FamilyTreeDNA does make the raw genetic data available for download and I asked my parents and brother to send me copies of their data.  If Paul and I are identical we should share almost 100% of our genetic code.  Furthermore, we should share significantly more with each other than with either of our parents with whom we should share on average about 50%.     

In [1]:
# Importing the libraries we'll use in this notebook.
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from functools import reduce

## Reading in the Data 


Ok, huge disclaimer - I'm not a geneticist or a biologist but this is what I think things are. 

The data files from FTDNA inludes:  

+ **RSID (Reference SNP ID)** - An SNP is a [single-nucleotide polymorphism](https://en.wikipedia.org/wiki/Single-nucleotide_polymorphism) which, as I understand it, are the nucleotides that vary from person to person and are not the same for every human being.

+ **Chromosome** - The chromosome where a given SNP is found. 

+ **Position** - The position on the chromosome where an SNP is located. 

+ **Result** - The paired allele values for the SNP. Not present when allele1/allele2 columns are provided. 

+ **allele1/allele2** - The individual values for alleles for a given SNP.  Not present when a result column is provided.  

In [2]:
# Reading in the csv files for each person.
scott = pd.read_csv('scott_dna.csv')
paul = pd.read_csv('paul_dna.csv')

mom = pd.read_csv('mom_dna.csv')
dad = pd.read_csv('dad_dna.csv')

# Peeking at our data. 

print("Scott\n",
      scott.head(), 
      '\n\nPaul\n', 
      paul.head(), 
      '\n\nMom\n', 
      mom.head(),
      '\n\nDad\n',
      dad.head())

/home/scott/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/scott/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Scott
        # name chromosome  position allele1 allele2
0   rs1042522         17   7579472       G       G
1   rs1057909         10  96741051       A       A
2   rs1175544          3  12467044       T       C
3  rs12721628          7  99367939       G       G
4  rs12721636          7  99381766       C       C 

Paul
        # name chromosome  position allele1 allele2
0   rs1042522         17   7579472       G       G
1   rs1057909         10  96741051       A       A
2   rs1175544          3  12467044       T       C
3  rs12721628          7  99367939       G       G
4  rs12721636          7  99381766       C       C 

Mom
          RSID CHROMOSOME  POSITION RESULT
0   rs4477212          1     82154     AA
1   rs3094315          1    752566     --
2   rs3131972          1    752721     GG
3  rs12562034          1    768448     --
4  rs12124819          1    776546     -- 

Dad
          RSID CHROMOSOME POSITION RESULT
0   rs4477212          1    82154     AA
1   rs3094315          1 

## Cleaning the Data 

As you can, the data files are formatted slightly differently so that Paul and I lack a "Result" column similar to that found in my parents data. However, we can construct the result column from the "allele1" and "allele2" column.  

Another issue is testing error.  For example, some SNPs are listed on the non-existant chromosome 0. Likewise, when an explicit error has been made the result is listed as "--".  In both cases we will drop these SNPs from the analysis. 

Finally, we'll relabel our columns so that we can tell individuals apart and have consistent naming conventions. 

In [3]:
# Let's create a combined result column like my parents' data.
scott['result'] = scott['allele1'] + scott['allele2']
paul['result'] = paul['allele1'] + paul['allele2']

# Drop out chromosomes that equal zero - this is a testing error.
scott.drop(scott[scott.chromosome == 0].index, inplace=True)
paul.drop(paul[paul.chromosome == 0].index, inplace=True)
mom.drop(mom[mom.CHROMOSOME == 0].index, inplace=True)
dad.drop(dad[dad.CHROMOSOME == 0].index, inplace=True)

# Drop cases where there is no result
mom.drop(mom[mom.RESULT == '--'].index, inplace=True)
scott.drop(scott[scott.result == '--'].index, inplace=True)
paul.drop(paul[paul.result == '--'].index, inplace=True)
dad.drop(dad[dad.RESULT == '--'].index, inplace=True)

# Drop out "famfinder" - combined file header
scott.drop(scott[scott['# name'] == '# famfinder'].index, inplace=True)
paul.drop(paul[paul['# name'] == '# famfinder'].index, inplace=True)

In [4]:
scott.columns = ["RSID",
                 "s_chromosome",
                 "s_position",
                 "s_allele1",
                 "s_allele2",
                 "s_result"]

paul.columns = ["RSID",
                "p_chromosome",
                "p_position",
                "p_allele1",
                "p_allele2",
                "p_result"]

mom.columns = ["RSID",
               "m_chromosome",
               "m_position",
               "m_result"]

dad.columns = ["RSID",
               "d_chromosome",
               "d_position",
               "d_result"]

## Joining Data 

Now we need to join the data so it can be uniformly compared.  We'll use my mothers data as a base and join in the basis of the RSID.   

In [5]:
dfs = [mom, dad, scott, paul]
df = reduce(lambda left, right: pd.merge(left, right, on='RSID'), dfs)

In [6]:
# Since dad won't share an X chromosome with us, let's drop it to put 
# both my parents on an equal footing. Only autosomal 

df.drop(df[df['m_chromosome'] == 'X'].index, inplace=True)
df.drop(df[df['d_chromosome'] == 'X'].index, inplace=True)
df.drop(df[df['p_chromosome'] == 'X'].index, inplace=True)
df.drop(df[df['s_chromosome'] == 'X'].index, inplace=True)

In [7]:
#Visually inspect the data - I want a few more rows than df.head() 
df[:10]

,RSID,m_chromosome,m_position,m_result,d_chromosome,d_position,d_result,s_chromosome,s_position,s_allele1,s_allele2,s_result,p_chromosome,p_position,p_allele1,p_allele2,p_result
0,rs4477212,1,82154,AA,1,82154,AA,1,82154,A,A,AA,1,82154,A,A,AA
1,rs3131972,1,752721,GG,1,752721,AG,1,752721,A,G,AG,1,752721,A,G,AG
2,rs11240777,1,798959,GG,1,798959,AG,1,798959,A,G,AG,1,798959,A,G,AG
3,rs4970383,1,838555,CC,1,838555,AC,1,838555,C,C,CC,1,838555,C,C,CC
4,rs4475691,1,846808,CC,1,846808,CC,1,846808,C,C,CC,1,846808,C,C,CC
5,rs7537756,1,854250,AG,1,854250,AA,1,854250,A,G,AG,1,854250,A,G,AG
6,rs13302982,1,861808,GG,1,861808,GG,1,861808,G,G,GG,1,861808,G,G,GG
7,rs1110052,1,873558,GT,1,873558,GT,1,873558,T,G,TG,1,873558,T,G,TG
8,rs2272756,1,882033,AG,1,882033,AG,1,882033,A,G,AG,1,882033,A,G,AG
9,rs17160698,1,887162,TT,1,887162,TT,1,887162,T,T,TT,1,887162,T,T,TT


# Calculation Percent Agreement 

Let's calculate raw percent agreement.  It should be noted this is **not how geneticists and genetic geneaologists calculate shared DNA** - it's going to over inflated shared DNA between unrelated individuals (**for instance, my mom and dad are not related!***).  But it will let us determine if my brother and I are identical.  

I am sure that there is a way to calculate chance adjusted agreement but this will answer our initial question.  

In [8]:
# Calculate agreement
df['s_p_agree'] = df['s_result'] == df['p_result']
df['s_m_agree'] = df['s_result'] == df['m_result']
df['s_d_agree'] = df['s_result'] == df['d_result']
df['p_m_agree'] = df['p_result'] == df['m_result']
df['p_d_agree'] = df['p_result'] == df['d_result']
df['m_d_agree'] = df['d_result'] == df['m_result']

In [9]:
# Create an array of comparisons 
pct_agree = np.array([
    [1, df['s_p_agree'].mean(), df['s_m_agree'].mean(), 
     df['s_d_agree'].mean()],
    [df['s_p_agree'].mean(), 1, df['p_m_agree'].mean(),
     df['p_d_agree'].mean()],
    [df['s_m_agree'].mean(), df['p_m_agree'].mean(),
     1, df['m_d_agree'].mean()],
    [df['s_d_agree'].mean(), df['p_d_agree'].mean(),
     df['m_d_agree'].mean(), 1]
])

In [ ]:
# Visualized the percent agreement data. 

names = ['Scott', 'Paul', "Mom", "Dad"]
fig, ax = plt.subplots()
im = ax.imshow(pct_agree, cmap=plt.cm.ocean)

ax.set_xticks(np.arange(len(names)))
ax.set_yticks(np.arange(len(names)))

ax.set_xticklabels(names)
ax.set_yticklabels(names)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(len(names)):
    for j in range(len(names)):
        text = ax.text(j, i, round(pct_agree[i, j], 2),
                       ha="center", va="center", color="black")

fig.tight_layout()
plt.title('Percent Agreement')
plt.show()

The above plot is pretty telling!  Paul and I are showing 100% agreement here on our base pairs.  That looks pretty identical.  However, one thing many people don't know is that identical twins are not truly identical - there are subtle mutations that mean that they are almost, but not quite, perfect copies.  Below I report the the number of base pairs on which we differed as well as how many by which I differed from my mother.   

In [ ]:
agreement = df.s_p_agree.value_counts().to_frame()
agreement["s_m_agree"] = df.s_m_agree.value_counts()
agreement

## Hamming Distance 

Another way to calculate difference is a hamming distance.  Hamming distance represents the number of positions that the strings differ from one another.  In this case, rather than looking at agreement between base pairs we're looking at agreement between alleles.   

In [ ]:
# Create strings of DNA for comparison. 
s = ''.join(list(df['s_result']))
p = ''.join(list(df['p_result']))
m = ''.join(list(df['m_result']))
d = ''.join(list(df['d_result']))

#let's check to make sure we're comparing apples to apples. 
print("DNA same length?:", len(s) == len(p) == len(m) == len(d), "\n") 

def hamming_distance(s1, s2):
    return sum(ch1 != ch2 for ch1,ch2 in zip(s1,s2))

# Calculate hamming distances. 
print("*** Hamming Distances between Individuals DNA ***")
print("Mom w/ Dad:    ", hamming_distance(m, d))
print("Mom w/ Scott:  ", hamming_distance(m, s))
print("Dad w/ Scott:  ", hamming_distance(d, s))
print("Mom w/ Paul:   ", hamming_distance(m, p))
print("Dad w/ Paul:   ", hamming_distance(d, p))
print("Scott w/ Paul: ", hamming_distance(s, p))

# Conclusion

So are Paul and I identical? Identical twins are *not truly identical* and there can be minute differences in DNA resulting from mutation or testing error.  The results here show this - my brother and I are not completely identical.  However, we differed on only 18 base pairs out of 659,179 and 21 alleles out of the 1,318,358 tested!  So we are absolutely identical twins.  We're far closer than typical siblings can be and are closer related to one another than we are to our parents. Considering how easy it is to verify this perhaps FamilyTreeDNA should include a "Self/Identical Twin" category.   